In [1]:
# file upload
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("homework").getOrCreate()

df_user_list = spark.read.csv("./data/user_list.csv", header=True, inferSchema=True)
df_detail_visit = spark.read.csv("./data/coupon_detail_train.csv", header=True, inferSchema=True)

# df_detail_visit.show()

In [2]:
df = df_user_list.join(df_detail_visit, "USER_ID_hash", "inner")
# df.show()

In [3]:
df.printSchema()

root
 |-- USER_ID_hash: string (nullable = true)
 |-- REG_DATE: timestamp (nullable = true)
 |-- SEX_ID: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- WITHDRAW_DATE: string (nullable = true)
 |-- PREF_NAME: string (nullable = true)
 |-- ITEM_COUNT: integer (nullable = true)
 |-- I_DATE: timestamp (nullable = true)
 |-- SMALL_AREA_NAME: string (nullable = true)
 |-- PURCHASEID_hash: string (nullable = true)
 |-- COUPON_ID_hash: string (nullable = true)



In [4]:
# create date col
from pyspark.sql.functions import date_format

df = df.withColumn("date", date_format(df["REG_DATE"], "yyyy-MM-dd"))
df.show()

+--------------------+-------------------+------+---+-------------+---------+----------+-------------------+----------------------------+--------------------+--------------------+----------+
|        USER_ID_hash|           REG_DATE|SEX_ID|AGE|WITHDRAW_DATE|PREF_NAME|ITEM_COUNT|             I_DATE|             SMALL_AREA_NAME|     PURCHASEID_hash|      COUPON_ID_hash|      date|
+--------------------+-------------------+------+---+-------------+---------+----------+-------------------+----------------------------+--------------------+--------------------+----------+
|d9dca3cb44bab12ba...|2012-03-28 14:14:18|     f| 25|           NA|     NULL|         1|2012-03-28 15:06:06|                        兵庫|c820a8882374a4e47...|34c48f84026e08355...|2012-03-28|
|560574a339f1b25e5...|2011-05-18 00:41:48|     f| 34|           NA|   東京都|         1|2011-07-04 23:52:54|      銀座・新橋・東京・上野|1b4eb2435421ede98...|767673b7a777854a9...|2011-05-18|
|560574a339f1b25e5...|2011-05-18 00:41:48|     f| 34|        

In [5]:
# select col
from pyspark.sql.functions import col

df = df.select(
    col("date"),
    col("USER_ID_hash").alias("user_id"),
    col("SEX_ID").alias("sex_id"),
    col("PREF_NAME").alias("name"),
    col("SMALL_AREA_NAME").alias("area_name"),
    col("PURCHASEID_hash").alias("purchased_id"),
    col("ITEM_COUNT").alias("item_count")
)

# df.show()

In [6]:
df.filter(df["name"].isNull()).show()

+----------+--------------------+------+----+----------------------------+--------------------+----------+
|      date|             user_id|sex_id|name|                   area_name|        purchased_id|item_count|
+----------+--------------------+------+----+----------------------------+--------------------+----------+
|2012-03-28|d9dca3cb44bab12ba...|     f|NULL|                        兵庫|c820a8882374a4e47...|         1|
|2012-02-08|43fc18f32eafb0571...|     m|NULL|                        山形|d88f6ab0f9b8d737c...|         1|
|2012-02-08|43fc18f32eafb0571...|     m|NULL|新宿・高田馬場・中野・吉祥寺|1cef71947aa1f80e7...|         1|
|2012-02-08|43fc18f32eafb0571...|     m|NULL|新宿・高田馬場・中野・吉祥寺|e47445e2f58d7e9c0...|         1|
|2012-02-08|43fc18f32eafb0571...|     m|NULL|新宿・高田馬場・中野・吉祥寺|0325b7df2461cc47c...|         1|
|2011-12-27|72f88db98f71be3f5...|     m|NULL|                      北海道|422c3e8172836560b...|         1|
|2011-12-27|72f88db98f71be3f5...|     m|NULL|新宿・高田馬場・中野・吉祥寺|8dd554e0e649cff7f...|     

In [7]:
# TODO : 현재는 item_count가 수치열인 것을 알고있지만 데이터가 많을 경우 타입을 확인하고 변경하는 것으로 로직 수정
null_value = dict()
for col in df.columns:
    if col == "item_count":
        null_value[col] = 0
    else:
        null_value[col] = "-"
# print(null_value)

df = df.fillna(null_value)
df.show()

+----------+--------------------+------+------+----------------------------+--------------------+----------+
|      date|             user_id|sex_id|  name|                   area_name|        purchased_id|item_count|
+----------+--------------------+------+------+----------------------------+--------------------+----------+
|2012-03-28|d9dca3cb44bab12ba...|     f|     -|                        兵庫|c820a8882374a4e47...|         1|
|2011-05-18|560574a339f1b25e5...|     f|東京都|      銀座・新橋・東京・上野|1b4eb2435421ede98...|         1|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|36b5f9ba46c44b655...|         1|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|2f30f46937cc90047...|         1|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|4d000c64a55ac573d...|         1|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|d8b030c8a4a2c1051...|         1|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|2c98138766edf5d5b..

In [8]:
df.filter(df["name"].isNull()).show()

+----+-------+------+----+---------+------------+----------+
|date|user_id|sex_id|name|area_name|purchased_id|item_count|
+----+-------+------+----+---------+------------+----------+
+----+-------+------+----+---------+------------+----------+



In [9]:
df = df.withColumn("cost", df["item_count"] * 8000) \
        .withColumn("cost_VAT", df["item_count"] * 8000 * 1.1)
df.show()

+----------+--------------------+------+------+----------------------------+--------------------+----------+-----+--------+
|      date|             user_id|sex_id|  name|                   area_name|        purchased_id|item_count| cost|cost_VAT|
+----------+--------------------+------+------+----------------------------+--------------------+----------+-----+--------+
|2012-03-28|d9dca3cb44bab12ba...|     f|     -|                        兵庫|c820a8882374a4e47...|         1| 8000|  8800.0|
|2011-05-18|560574a339f1b25e5...|     f|東京都|      銀座・新橋・東京・上野|1b4eb2435421ede98...|         1| 8000|  8800.0|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|36b5f9ba46c44b655...|         1| 8000|  8800.0|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|2f30f46937cc90047...|         1| 8000|  8800.0|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|4d000c64a55ac573d...|         1| 8000|  8800.0|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒

In [10]:
df.printSchema()

root
 |-- date: string (nullable = false)
 |-- user_id: string (nullable = false)
 |-- sex_id: string (nullable = false)
 |-- name: string (nullable = false)
 |-- area_name: string (nullable = false)
 |-- purchased_id: string (nullable = false)
 |-- item_count: integer (nullable = false)
 |-- cost: integer (nullable = false)
 |-- cost_VAT: double (nullable = false)



In [11]:
# create all date cols
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, weekofyear

df = df.withColumn("date", to_date(df.date, "yyyy-MM-dd"))

df = df.withColumn("year", year(df.date)) \
    .withColumn("month", month(df.date)) \
    .withColumn("day", dayofmonth(df.date)) \
    .withColumn("dow", dayofweek(df.date)) \
    .withColumn("week", weekofyear(df.date))
df.show()

+----------+--------------------+------+------+----------------------------+--------------------+----------+-----+--------+----+-----+---+---+----+
|      date|             user_id|sex_id|  name|                   area_name|        purchased_id|item_count| cost|cost_VAT|year|month|day|dow|week|
+----------+--------------------+------+------+----------------------------+--------------------+----------+-----+--------+----+-----+---+---+----+
|2012-03-28|d9dca3cb44bab12ba...|     f|     -|                        兵庫|c820a8882374a4e47...|         1| 8000|  8800.0|2012|    3| 28|  4|  13|
|2011-05-18|560574a339f1b25e5...|     f|東京都|      銀座・新橋・東京・上野|1b4eb2435421ede98...|         1| 8000|  8800.0|2011|    5| 18|  4|  20|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|36b5f9ba46c44b655...|         1| 8000|  8800.0|2011|    5| 18|  4|  20|
|2011-05-18|560574a339f1b25e5...|     f|東京都|          恵比寿・目黒・品川|2f30f46937cc90047...|         1| 8000|  8800.0|2011|    5| 18|  4|  20|
|201

In [12]:
numeric_cols = ["item_count", "cost", "cost_VAT"]
category_cols = df.columns

category_cols = list(set(category_cols) - set(numeric_cols))
category_cols

['week',
 'dow',
 'date',
 'month',
 'user_id',
 'sex_id',
 'day',
 'year',
 'purchased_id',
 'name',
 'area_name']

In [13]:
from pyspark.sql.functions import sum

df = df.groupby(category_cols).agg(
    sum("item_count").alias("item_count"),
    sum("cost").alias("cost"),
    sum("cost_VAT").alias("cost_VAT")
)
df.show()

+----+---+----------+-----+--------------------+------+---+----+--------------------+--------+----------------------------+----------+-----+------------------+
|week|dow|      date|month|             user_id|sex_id|day|year|        purchased_id|    name|                   area_name|item_count| cost|          cost_VAT|
+----+---+----------+-----+--------------------+------+---+----+--------------------+--------+----------------------------+----------+-----+------------------+
|  52|  1|2012-01-01|    1|409d495d62c15f6fd...|     m|  1|2012|cc3db35dcb7b87ed8...|  奈良県|                        キタ|         2|16000|           17600.0|
|   2|  2|2012-01-09|    1|eb2e1864a5db99653...|     f|  9|2012|163776c175bfc7d6a...|  東京都|新宿・高田馬場・中野・吉祥寺|         1| 8000|            8800.0|
|  32|  4|2011-08-10|    8|016d9e22f328423dc...|     m| 10|2011|08317e14836733ff3...|  奈良県|                        山形|         1| 8000|            8800.0|
|  25|  4|2011-06-22|    6|b513d24721f5f8628...|     m| 22|2011|3d1

In [14]:
output_path = "./data/result"
df.write.parquet(output_path)

In [15]:
saved_df = spark.read.parquet(output_path)
saved_df.show()

+----+---+----------+-----+--------------------+------+---+----+--------------------+--------+----------------------------+----------+-----+--------+
|week|dow|      date|month|             user_id|sex_id|day|year|        purchased_id|    name|                   area_name|item_count| cost|cost_VAT|
+----+---+----------+-----+--------------------+------+---+----+--------------------+--------+----------------------------+----------+-----+--------+
|  20|  1|2011-05-22|    5|dc6df8aa860f8db0d...|     m| 22|2011|9294198d4e3e40aa9...|神奈川県|          川崎・湘南・箱根他|         2|16000| 17600.0|
|  37|  3|2011-09-13|    9|a96643bfe2d011d30...|     m| 13|2011|080a3bdaa26656e1e...|神奈川県|新宿・高田馬場・中野・吉祥寺|         1| 8000|  8800.0|
|  45|  1|2010-11-14|   11|2a1005b213cd0e051...|     f| 14|2010|8ee80710133f5d386...|  三重県|                        奈良|         1| 8000|  8800.0|
|   7|  1|2012-02-19|    2|fe213bdfeb9e617b1...|     f| 19|2012|f747306ac8fd48c5a...|  福岡県|新宿・高田馬場・中野・吉祥寺|         1| 8000|  8800.0|
|  